In [1]:
####################### USER INPUTS #####################################

#User must input a list of txt files to calculate b-value order
#Format of input txt files: 
# column 1: loc 1(x, lat, etc)
# column 2: loc 2(y, lon, etc) ## These will not be used in any calculationin this script
# these columns may be omitted but user will have to change column numbers in later part of script
# column 3: depth (in km)
# column 4: ml (or other magnitude value)
# column 5: ID #
# column 6: year

#Note, if the order of these columns is rearranged, we can work with this, will need to change column numbers in other parts of code

# user must also input the start_years for each file. This is calculated in separate program

file=['associated1','associated2','associated3','associated4','associated5','associated6','associated7','associated8','associated9','associated10','associated11','associated12','associated13','associated14','associated15','associated16','associated17','associated18']
start_years=[2006.0, 2006.0, 2005.0, 2004.0, 2004.0, 2005.0, 2008.0, 2006.0, 2006.0, 2006.0, 2006.0, 2006.0, 2006.0, 2006.0, 2006.0, 2006.0, 2006.0, 2006.0]
depth_limit=50
#shallow_depth_limit=50
#deep_depth_limit=100

# This is the end of the base needed user inputs but take note of other 
# possible changes that a user can make and where to adjust:
# test range of minimum magnitude values:
# test range of maximum magnitude values:
# transect_number values(changes title label in plot images):
####################### END OF USER INPUTS #############################

#import the modules that are necessary for the subsequent calculations 
#and commands
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math
import random
import csv
import matplotlib as mpl
# Set transect numbers (This will be labels used in plots to differentiate between files)
# can change to a label instead (i.e. 'black peak' instead of '12')
# must replace next two lines with list of strings for labels,  if change is wanted 
transect_number=np.arange(1,len(file)+1,1)
transect_number=np.ndarray.tolist(transect_number)

lower_cutoff_magnitude, upper_cutoff_magnitude, final_lower_cutoff_magnitude=[],[],[]
depth_arrays, ml_arrays, y_arrays, B_arrays, bin_edges_arrays=[],[],[],[],[]

#range of values considered for the maximum magnitude cutoff range and minimum magnitude 
#cutoff range respectively, these values may be changed as needed. Lowest value considered to 
# be 2.0 in this example because no records of events w/ magnitude < 2.0 are recorded for 
# AEC catalog
max_mag=np.arange(3.5,7.0,0.1)
min_mag=np.arange(2.0,3.1,0.1)

############################################################################################
############################################################################################

# open files and extract columns of data as lists to use later. This is where parameters
# should be changed if data column format does not match the original input format
for i in range(len(file)):
    start=start_years[i]
    transect=transect_number[i]
    d,depth,m,ml,years=[],[],[],[],[]
    evtfl=open(file[i])
    for line in evtfl:
        d.append(line.split()[2]) # <--- depth column
        m.append(line.split()[3]) # <--- magnitude column
        years.append(line.split()[5]) # <--- year column 
    
    #keep only the data that is from after the start point given in the list above and only everything deeper than 50 km

    years=[float(i) for i in years]
    for i in range(len(years)):
        if years[i]<start:
            continue
        else:
            depth.append(d[i])
            ml.append(m[i])
            
    depth=[float(i) for i in depth]
    ml=[float(i) for i in ml]
    ml=[ml[i] for i in range(len(depth)) if depth[i]>=depth_limit]
    depth=[depth[i] for i in range(len(depth)) if depth[i]>=depth_limit]
    #ml=[ml[i] for i in range(len(depth)) if depth[i]>=shallow_depth_limit and depth[i]<= deep_depth_limit]    
    #depth=[depth[i] for i in range(len(depth)) if depth[i]>=shallow_depth_limit and depth[i]<= deep_depth_limit]
    depth_arrays.append(depth)
    ml_arrays.append(ml)

    # create histogram and cumulative histogram to plot real data points as frequency vs. magnitude bins

    binwidth=0.1
    Nbins=np.arange(2.0, max(ml)+binwidth, binwidth)
    hist,bin_edges=np.histogram(ml,bins=Nbins)
    bin_edges=bin_edges[:-1]
    cum_y=[sum(hist[i:]) for i in range(len(hist))]
    y=np.log10(cum_y)
    
    y_arrays.append(y)
    B_arrays.append(y)
    bin_edges_arrays.append(bin_edges)
    
############################################################################################
############################################################################################
# functions for use later 

#subroutine for generating synthetic data, should only be used for first step b/c of the 
# way that the first step loops differently than second and third to create all of the lines

def generate_synthetic_data_step1(ml,binwidth,bval,aval):

    syn_ml=np.arange(2.0, max(ml)+binwidth, binwidth)
    syn_ml=syn_ml[1:]
    synthetics=[]
    for i in range(len(aval)):
        syn_data=bval[i]*-1*syn_ml+aval[i]
        synthetics.append(syn_data)   
        
    return syn_data,synthetics

#subroutine for generating plots for second and third steps 
def plots(residuals,LMC,UMC,bin_edges,bins_,synthetics,transect,outfile_s,outfile_r):

            bin_edges=np.ndarray.tolist(bin_edges)
            bin_edges=[round(n,1) for n in bin_edges]
            
            j=bin_edges.index(LMC)
            k=bin_edges.index(UMC)

            plt.figure()
            
            plt.plot(bin_edges,y,color='lightgray', marker='o', linestyle='None')
            plt.plot(bin_edges[j:k+1],y[j:k+1],'co')
            plt.plot(bins_[i],synthetics[i],'k')
            plt.title('Transect'+str(transect))
            plt.xlabel('Ml')
            plt.ylabel('Log N(M)')
            plt.ylim(-0.1,3.2)
            #plt.savefig('final_LMC_synthetics_transect'+str(transect)+'_50.jpg')
            plt.savefig(outfile_s)
            plt.figure()
            plt.plot(residuals_bins,residuals,'r^')
            plt.plot(residuals_bins,residuals, 'r-')
            plt.title('Residuals for transect '+str(transect))
            plt.xlabel('LMCs')
            plt.ylabel('Residuals in %')
            plt.ylim(0,27.5)
            #plt.savefig('final_LMC_residuals_transect'+str(transect)+'_50.jpg')
            plt.savefig(outfile_r)

            
def maximum_likelihood(ml_cut, mc):
    mu=sum(ml_cut)/len(ml_cut) 
    m=0.1
    p=1+(m/(mu-mc))
    b=(1/(math.log(10)*m))*math.log(p)
    bval.append(b)
    a = np.log10(len(ml_cut)) + b* mc
    aval.append(a)
    return a,b 
                
def residuals_calc( bins, B, syn_data):
    bins_.append(bins)
    real.append(B)
    R=[]  

    r= abs(B-syn_data)
    R.append(r)
    resid=((sum(R)/sum(B))*100)
    residuals.append(sum(resid))
    
############################################################################################
################################# STEP 1 ###################################################

for i in range(len(file)):
    ml=ml_arrays[i]
    y=y_arrays[i]
    B=B_arrays[i]
    bin_edges=bin_edges_arrays[i]
    synthetics=[]
    transect=transect_number[i]

    aval,b,bval=[],[],[]
    for j in range(len (min_mag)):
        ycut,bin_edgescut=[],[]
        ml_cut=[]
        for i in range (len(y)):
            if bin_edges[i]>=min_mag[j] and bin_edges[i]<5.0:
                ycut.append(y[i])
                bin_edgescut.append(bin_edges[i])
        for i in range (len(ml)):
            if ml[i]>=min_mag[j]:
                ml_cut.append(ml[i])
                
        mc=min_mag[j]
        maximum_likelihood(ml_cut,mc)
        syn_data,synthetics=generate_synthetic_data_step1(ml,binwidth,bval,aval)
        

    B=y
    residuals=[]
    for i in range(len(synthetics)):
        R=[]
        for j in range(len(synthetics[i])):
            S=synthetics[i]
            r= abs(B[j]-S[j])
            R.append(r)
        resid=((sum(R)/sum(B))*100)
        residuals.append(resid)
    
    plt.figure()
    mpl.rcParams['pdf.fonttype'] = 42
    mpl.rcParams['font.size']=16
    plt.plot(bin_edges,y, 'co')
    
    for i in range(len(synthetics)):
        plt.plot(bin_edges, synthetics[i])

    plt.plot(bin_edges, y,'bo') 
    plt.title('Possible LMC Fits, Synthetics and Real Data, Transect '+str(transect))
    plt.ylabel('Log N(M)')
    plt.xlabel('Ml')
    plt.savefig('LMC_synthetics_transect'+str(transect)+'_50.jpg')
    plt.savefig('LMC_synthetics_transect'+str(transect)+'_50.pdf')

    plt.figure()
    plt.plot(min_mag, residuals, 'r^')
    plt.plot(min_mag, residuals, 'r-')
    plt.title('Residuals for possible LMCs, transect '+str(transect))
    plt.xlabel('LMCs')
    plt.ylabel('Residuals in %')
    plt.savefig('LMC_residuals_transect'+str(transect)+'_50.jpg')
    plt.savefig('LMC_residuals_transect'+str(transect)+'_50.pdf')
           
    for i in range(len(residuals)):
        if residuals[i]<= 10:
            lower_cutoff_magnitude.append(min_mag[i])
            break
            
        elif residuals[i]== min(residuals):
            lower_cutoff_magnitude.append(min_mag[i])
            
###########################################################################################
################################ STEP 2 ###################################################

for i in range(len(file)):
    ml=ml_arrays[i]
    y=y_arrays[i]
    B=B_arrays[i]
    bin_edges=bin_edges_arrays[i]
    transect=transect_number[i]
    mc=lower_cutoff_magnitude[i]
    LMC=round(lower_cutoff_magnitude[i],1)
    
    
    #now here comes the tricky part, we need to trucate for each minimum magnitude but leaving the high magnitude
    # in tact and then calculate the linear regression coefficients and store them
    aval,b,bval,synthetics, real, bins_=[],[],[],[],[],[]
    syn_ml=np.arange(mc,3.5,0.1)
    cutoffs=[]
    residuals=[]
    r_bins=[]
    for j in range(len (max_mag)):
        ml_cut=[]

        for i in range (len(ml)):
            if ml[i]<=max_mag[j] and ml[i]>=mc:
                ml_cut.append(ml[i])

        a,b= maximum_likelihood(ml_cut, mc)
        syn_ml=np.ndarray.tolist(syn_ml)
        
        if max_mag[j] > max(bin_edges):
            break
            
        syn_ml.append(max_mag[j])
        syn_ml=np.asarray(syn_ml)
        syn_data=b*-1*syn_ml+a
        synthetics.append(syn_data)


        B,bins=[],[]

        for i in range(len(bin_edges)):

            if bin_edges[i] >= mc and bin_edges[i] <= max_mag[j]+0.01:
                B.append(y[i])
                bins.append(round(bin_edges[i],1))        

        residuals_calc( bins, B, syn_data)
    residuals_bins=np.arange(3.5,max(bin_edges),0.1)
    
    for i in range(len(residuals)):  
        if residuals[i]== min(residuals):
            upper_cutoff_magnitude.append(max_mag[i])
            UMC=round(max_mag[i],1)
            plots(residuals,LMC,UMC,bin_edges,bins_,synthetics,transect,'UMC_synthetics_transect '+str(transect)+'_50.jpg','UMC_residuals_transect'+str(transect)+'_50.jpg')

umc=[round(i,1) for i in upper_cutoff_magnitude]

###########################################################################################
################################### STEP 3 ################################################

for i in range(len(file)):
    ml=ml_arrays[i]
    y=y_arrays[i]
    B=B_arrays[i]
    bin_edges=bin_edges_arrays[i]
    transect=transect_number[i]
    mc=lower_cutoff_magnitude[i]
    transect=transect_number[i]
    start=start_years[i]
    UMC=umc[i]
        
    aval,b,bval,synthetics,real,bins_ = [],[],[],[],[],[]
                      
    cutoffs=[]
    residuals=[]
    r_bins=[]
    
    for j in range(len(min_mag)):
        ml_cut=[]
        syn_ml=np.arange(min_mag[j],UMC+binwidth,0.1) 
        
        for i in range(len(ml)):
            if ml[i]>= min_mag[j] and ml[i]<=UMC:
                ml_cut.append(ml[i])
                
        mu= sum(ml_cut)/len(ml_cut) #sample average
        delta_m=0.1                 #magnitude interval
        
        p=1+(delta_m/(mu-min_mag[j]))
        b=(1/(math.log(10)*delta_m))*math.log(p)
        
        bval.append(b)
        
        a= np.log10(len(ml_cut))+b*min_mag[j]
        aval.append(a)
        
        syn_data=b*-1*syn_ml+a
        synthetics.append(syn_data)
        
        B,bins=[],[]
        for i in range(len(bin_edges)):
            if bin_edges[i] >= min_mag[j] and bin_edges[i]<= UMC+0.1:
                B.append(y[i])
                bins.append(round(bin_edges[i],1))
                

        residuals_calc( bins, B, syn_data)
    residuals_bins=np.arange(2.0,3.1,0.1)
    
    for i in range (len(residuals)):
        if residuals [i] == min(residuals):
            final_lower_cutoff_magnitude.append(min_mag[i])
            LMC=round(min_mag[i],1)
            
            plots(residuals,LMC,UMC,bin_edges,bins_,synthetics,transect,'final_LMC_synthetics_transect'+str(transect)+'_50.jpg','final_LMC_residuals_transect'+str(transect)+'_50.jpg')
            
############################################################################################
############################################################################################
error, amt, aval,b,bval,error,eventsN=[],[],[],[],[],[],[]
for i in range(len(file)):
    
    mc=final_lower_cutoff_magnitude[i]
    m_max=upper_cutoff_magnitude[i]
    
    start=start_years[i]
    #open file and create empty lists for the info that is needed
    d,depth,m,ml,years=[],[],[],[],[]
    evtfl=open(file[i])

    #fill the empty lists with data from the associated file
    for line in evtfl:
        d.append(line.split()[2])
        m.append(line.split()[3])
        years.append(line.split()[5])
        
    years=[float(i) for i in years]
    
    #keep only the data that is from after the start point given in the list above
    for i in range(len(years)):
        if years[i]<start:
            continue
        else:
            depth.append(d[i])
            ml.append(m[i])
     
    #change data type from strings to floating point numbers
    depth=[float(i) for i in depth]
    ml=[float(i) for i in ml]
    
    #set up a list of values for possible minimum magnitudes
    #we will carry out linear regression on each of these in order to get estimates for what should be the 
    #actual magnitude of completeness
    max_mag=np.arange(3.5,6.6,0.1)
    #keep only the data that is deeper than 50 km
    ml=[ml[i] for i in range(len(depth)) if depth[i]>=50]
    depth=[depth[i] for i in range(len(depth)) if depth[i]>=50]

    amt.append(len(depth))
    
    #set up bins for histogram sorting *note that Weimer and Wyss use cumulative magnitude for their methods
    binwidth=0.1
    Nbins=np.arange(2.0, max(ml)+binwidth, binwidth)
    #create histogram from real data
    hist,bin_edges=np.histogram(ml,bins=Nbins)
    bin_edges=bin_edges[:-1]
    #make into a cumulative histogram
    cum_y=[sum(hist[i:]) for i in range(len(hist))]
    #take the log10 of this distribution
    y=np.log10(cum_y)
    
    #now here comes the tricky part, we need to trucate for each minimum magnitude but leaving the high magnitude
    # in tact and then calculate the linear regression coefficients and store them
    
    ml_cut=[]
    for j in range (len(ml)):
        if ml[j]>=mc and ml[j]<= m_max:
            ml_cut.append(ml[j])
                
        ################################################################################
        ######################## MAXIMUM LIKELIHOOD ####################################
        ################################################################################
        # ycut= y values for log10(cum_y)
        # bin_edgescut= bin values
        
    average=sum(ml_cut)/len(ml_cut) 
    m=0.1
    
    p=1+(m/(average-mc))
    b_=(1/(math.log(10)*m))*math.log(p)
    bval.append(b_)

    a = np.log10(len(ml_cut)) + b_* mc
    aval.append(a)
    
    eventsN.append(len(ml_cut))
    
    


    
    depth=[depth[i] for i in range(len(depth)) if ml[i]>=mc and ml[i]<= m_max]
    ml=[ml[i] for i in range(len(ml)) if ml[i]>= mc and ml[i] <= m_max]


    n=0
    nmax=1000
    boot_b=[]
    while n<nmax:
        index=random.sample(range(len(depth)),round(0.9*len(depth)))

        #use these index numbers to make a list of new magnitude data
        boot_ml=[]
        for i in range(len(index)):
            x=index[i]
            boot_ml.append(ml[x-1])

        binwidth=0.1
        Nbins=np.arange(2.0, max(ml)+binwidth, binwidth)
        #create histogram from real data
        hist,bin_edges=np.histogram(ml,bins=Nbins)
        bin_edges=bin_edges[1:]
        #make into a cumulative histogram
        cum_y=[sum(hist[i:]) for i in range(len(hist))]
        #take the log10 of this distribution
        y=np.log10(cum_y)

        #now here comes the tricky part, we need to trucate for each minimum magnitude but leaving the high magnitude
        # in tact and then calculate the linear regression coefficients and store them

        boot_ml_cut=[]
        for j in range (len(boot_ml)):
            if boot_ml[j]>=mc and boot_ml[j]<= m_max:
                boot_ml_cut.append(boot_ml[j])

            ################################################################################
            ######################## MAXIMUM LIKELIHOOD ####################################
            ################################################################################
            # ycut= y values for log10(cum_y)
            # bin_edgescut= bin values

        average=sum(boot_ml_cut)/len(boot_ml_cut) 
        m=0.1

        p=1+(m/(average-mc))
        b_=(1/(math.log(10)*m))*math.log(p)

        a = np.log10(len(boot_ml_cut)) + b_* mc

        boot_b.append(b_)

        n+=1

    bootstrapped_b=np.mean(boot_b)
    bootstrapped_std=np.std(boot_b)

    error.append(bootstrapped_std*2)

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.size']=12
mpl.rcParams['font.family'] = 'Arial'
plt.errorbar(transect_number, bval, yerr=error, xerr=None, fmt='r-')
plt.plot(transect_number, bval, 'r^')
plt.xticks([1,3,5,7,9,11,13,15,17],['1','3','5','7','9','11','13','15','17'])
plt.xlabel('Transect Number')
plt.ylabel('b-value')
plt.ylim(0.72,1.42)
plt.title('b-Values After Bootstrapping '+str(nmax)+' times')
plt.savefig('b_values_bootstrap_50.jpg')
plt.savefig('b_values_bootstrap_50.pdf')
    
print(bval)
plt.figure()
plt.plot(transect_number, bval, 'ro')
plt.plot(transect_number, bval, 'r-')
plt.xticks([1,3,5,7,9,11,13,15,17],['1','3','5','7','9','11','13','15','17'])
plt.title('Calculated b-Value')
plt.ylim(0.8,1.42)
plt.ylabel('b')
plt.xlabel('Transect Number')
plt.tight_layout()
plt.savefig('b-Value_50.jpg')
plt.savefig('b-Value_50.pdf')

plt.figure()
plt.plot(transect_number, final_lower_cutoff_magnitude)
plt.plot(transect_number, upper_cutoff_magnitude)
plt.xticks([1,3,5,7,9,11,13,15,17],['1','3','5','7','9','11','13','15','17'])
plt.xlabel('Transect Number')
plt.ylabel('Cutoffs')
plt.title('Magnitude Cutoffs for Each Transect')
plt.tight_layout()
plt.savefig('cutoffs_50.jpg')
plt.savefig('cutoffs_50.pdf')







/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:210: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:225: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:210: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:129: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:140: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:129: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:140: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:129: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:140: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:140: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:129: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:140: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max

[0.9957226022948604, 1.35767250782096, 1.3504779050640523, 1.2577004032611583, 1.1348632266296799, 0.8704840782577989, 0.9307290416122707, 1.1505084107036714, 1.141944621617096, 1.1809931207799498, 1.0423781107236074, 0.8602808934615511, 0.9326690992231116, 0.9136506608859102, 0.9407807192277127, 1.0108926864994419, 1.0070730191766053, 0.984062411236048]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:515: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:527: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
